# About The Company
Bellabeat, a smart device manufacture company especially designed for woman. Bellabeat is a successfull small company in smart device segments.Urška Sršen, cofounder and Chief Creative Officer of Bellabeat, believes that analyzing smart device fitness data could help unlock new growth opportunities for the company.
   We will move further step by step. There are total 5 steps. Ask, Prepare, Process, Analysis And Share. So, at First we will interpret what is business task or what is the problem we are going to solve.
#  ASK PHASE 
Sršen knows that an analysis of Bellabeat’s available consumer data would reveal more opportunities for growth. Identify tends in how consumers use non-Bellabet smart devices to apply insights into Bellabeat marketing strategy.She then wants you to select one Bellabeat product to apply these insights to in your presentation.
  
**Stakeholder's**
  
* Urška Sršen: Bellabeat’s cofounder and Chief Creative Officer
* Sando Mur: Mathematician and Bellabeat’s cofounder; key member of the Bellabeat executive team

# PREPARE PHASE
  Now, first we gather Fitbit Fitbase Tracker Data and load packages and libraries.

In [ ]:
install.packages("tidyverse")
install.packages("lubridate")
install.packages("skimr")
install.packages("janitor")
install.packages("ggplot2")
library(tidyverse)
library(lubridate)
library(dplyr)
library(ggplot2)
library(tidyr)
library(skimr)
library(janitor)

In [ ]:
daily_activity <- read_csv(file = "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv") 
daily_calories <- read_csv(file = "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/dailyCalories_merged.csv") 
intensities <- read_csv(file = "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/hourlyIntensities_merged.csv") 
daily_steps <- read_csv(file = "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/dailySteps_merged.csv") 
sleepday <- read_csv(file = "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv") 
weight <- read_csv(file = "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/weightLogInfo_merged.csv")

In [ ]:
head(daily_activity)

In [ ]:
head(daily_calories)

In [ ]:
head(intensities)

In [ ]:
head(daily_steps)

In [ ]:
head(sleepday)

In [ ]:
head(weight)

# PROCESS PHASE
We will check for errors in datasets and transform data.

In [3]:
glimpse(daily_activity)
glimpse(daily_calories)
glimpse(intensities)
glimpse(daily_steps)
glimpse(sleepday)
glimpse(weight)

ERROR: Error in glimpse(daily_activity): could not find function "glimpse"


The date and time column are merged. So, we need to separate them

In [ ]:
#separating date and time column
intensities$ActivityHour=as.POSIXct(intensities$ActivityHour, format="%m/%d/%Y %I:%M:%S %p", tz=Sys.timezone())
intensities$date <- format(intensities$ActivityHour, format = "%m/%d/%y")
intensities$time <- format(intensities$ActivityHour, format = "%H:%M:%S")


sleepday$SleepDay=as.POSIXct(sleepday$SleepDay, format="%m/%d/%Y %I:%M:%S %p", tz=Sys.timezone())
sleepday$date <- format(sleepday$SleepDay, format = "%m/%d/%y")
sleepday$time <- format(sleepday$SleepDay, format = "%H:%M:%S")

head(intensities)
head(sleepday)

Now we check for any duplicates

In [ ]:
sum(duplicated(daily_activity))
sum(duplicated(sleepday))
sum(duplicated(daily_steps))
sum(duplicated(intensities))
sum(duplicated(daily_calories))
sum(duplicated(weight))

There are 3 duplicates in sleepday dataset

In [ ]:
sleepday <- sleepday %>%
  distinct() %>%
  drop_na()
sum(duplicated(sleepday))

Now, our data is cleaned and tranformed. We checked for duplicate, data formats, timestamps.

# ANALYSIS PHASE
We cleaned the dataset and now we can start our analysis phase

In [ ]:
length(unique(daily_activity$Id))
length(unique(daily_calories$Id))
length(unique(intensities$Id))
length(unique(daily_steps$Id))
length(unique(sleepday$Id))
length(unique(weight$Id))

There are intotal 33 participants. But in sleepday participants are 24 and for weightlog dataset there are only 8 participants which concludes data is insufficiant which leads to biased recommendations and conclusions.

In [ ]:
#activity
daily_activity %>%
 select(TotalSteps, TotalDistance, SedentaryMinutes, Calories) %>%
summary()
#num of active minute per category 
daily_activity %>%
select (VeryActiveMinutes, FairlyActiveMinutes, LightlyActiveMinutes)%>%
summary()
#calories
daily_calories %>%
select(Calories)%>%
summary()
#intensities
intensities %>%
select(TotalIntensity, AverageIntensity)%>%
summary()
#steps
daily_steps %>%
select(StepTotal)%>%
summary()
#sleep
sleepday %>%
select(TotalMinutesAsleep,TotalTimeInBed)%>%
summary()
#weight
weight %>%
select(WeightKg, BMI)%>%
summary()

From summary 
* average sedentary minutes are 991.2 or 16.52 hours which is higher than usual.
* The majority of the participants are lightly active
* According CDC or doctor recommendation daily total steps should be 10,000 steps or higher and participants average steps are 7638 which is slightly lesser.
* Total time in bed is higher than total minutes sleep in bed which concludes rest time is little bit higher.

In [ ]:
summary(weight$IsManualReport)

There are 26 false manual reports so dataset of weight log is incorrectly recorded.

In [ ]:
daily_activity <- daily_activity %>%
rename(date = ActivityDate)%>%
mutate(date = as_date(date, format = "%m/%d/%Y"))
head(daily_activity)

In [ ]:
sleepday <- subset(sleepday, select = -date)
head(sleepday)

In [ ]:
sleepday <- sleepday %>%
rename(date = SleepDay)
head(sleepday)

In [ ]:
# merging datasets
daily_activity_merged <- daily_activity %>%
merge(sleepday, by = c("Id", "date"))
head(daily_activity_merged)

In [ ]:
daily_average <- daily_activity_merged %>%
group_by(Id) %>%
summarise (mean_daily_steps = mean(TotalSteps), mean_daily_calories = mean(Calories), mean_daily_sleep = mean(TotalMinutesAsleep))
head(daily_average)

In [ ]:
active_type <- daily_average %>%
  mutate(active_type = case_when(
    mean_daily_steps < 5000 ~ "sedentary",
    mean_daily_steps >= 5000 & mean_daily_steps < 7499 ~ "lightly active", 
    mean_daily_steps >= 7500 & mean_daily_steps < 9999 ~ "fairly active", 
    mean_daily_steps >= 10000 ~ "very active"
  ))

head(active_type)

In [ ]:
calories <- daily_activity %>%
group_by(Id)%>%
summarise(total_calories = sum(Calories))
head(calories)

As per doctors recommendation monthly calories should be burned **60,000** to **75,000**
Calories burned are depends upon person various factors like Sex, Age, Weight, Height and lifestyle.
From above table monthly calories **lesser than usual** are burned is observed. **Bellabeat app can notify them to do more activity
so they can burn more calories to achieve monthly goal. Bellabeta App can notify them on daily basis and monthly basis.**

# SHARE PHASE
The last phase of our case study is visualization phase.

In [ ]:
ggplot(data = daily_activity_merged) +
geom_point(mapping = aes(x = TotalSteps, y = Calories)) + 
geom_smooth (mapping = aes(x = TotalSteps, y = Calories)) +
labs(title = "Total Steps Vs Calories")

I can see there is a **positive co-relation** between total steps and calories which means more the steps more calories will burn.
According to doctors on an average **1600 to 2200** calories should be burn daily for women. **Bellabeat app can recommend complete daily goal at least 9000 to 10,000 steps to burn adequate amount of calories.**
Now, let's check at what time participant is most active

In [ ]:
int_new <- intensities %>%
group_by(time) %>%
drop_na() %>%
summarise(mean_total_int = mean(TotalIntensity))
head(int_new)

In [ ]:
intensities <- subset(intensities, select = -ActivityHour)
head(intensities)

In [ ]:
intensities <- intensities %>%
group_by(Id) %>%
drop_na()%>%
summarise(mean_total_int = mean(TotalIntensity)) %>%
mutate(intense_type = case_when (
    mean_total_int < 5 ~ "Sedentary",
    mean_total_int >= 5 & mean_total_int < 10 ~ "Lightly intense", 
    mean_total_int >= 10 & mean_total_int < 15 ~ "Fairly intense", 
    mean_total_int >= 15 ~ "Highly Intense"
  ))
head(intensities)

In [ ]:
ggplot(data = int_new, aes(x = time, y = mean_total_int)) + 
geom_histogram(stat = "identity", fill = 'darkgreen') +
theme(axis.text.x = element_text(angle = 90)) +
labs(title = "Average Total Intensity vs. Time")

From above graph participants are active during 5 am to 10 pm.
The participants are **mostly active** during **5 pm** to **10 pm**. I think
they prefer to go gym or for jogging after office work.
**Bellabeat app can use this time to remind and motivate them to do excersize.**

In [ ]:
ggplot(data = sleepday) +
geom_point (mapping = aes(x = TotalMinutesAsleep, y = TotalTimeInBed)) + 
geom_smooth(mapping = aes(x = TotalMinutesAsleep, y = TotalTimeInBed)) + 
labs(title = "Total Minutes Asleep  vs.Total Time in Bed")

The relationship between Total Minutes Asleep and Total Time in Bed looks **linear**. So if the Bellabeat users want to improve their sleep, we should consider using notification to go to sleep.

In [ ]:
ggplot(data = daily_activity_merged, aes(x = TotalMinutesAsleep, y = SedentaryMinutes)) + 
geom_point(color = 'darkgreen') + geom_smooth(color = 'red') +
labs(title="Minutes Asleep vs. Sedentary Minutes")

# Summerizing Data and Meaningful Insights
As we know Bellabeat company founded in 2013 and it has grown rapidly and quickly positioned itself as a tech-driven wellness company for women. Bellabeat company especially designed for women and founder goal is to connect and improve women health through technology.Some type of diseases   are more prone to women then men. So, Urška Sršen want to monitor and improve health through smart devices. Here I found some meaningful insight's
to achive goal and enhance company credibility.
1. If the Bellabeat users want to improve their sleep, we should consider using notification to go to sleep.
2. According CDC or doctor recommendation daily total steps should be 10,000 steps or higher and participants average steps are 7638 which is slightly lesser. Bellabeat app can notify users about total daily steps and suggest daily steps according to user goal.
3. Bellabeat app can recommend complete daily goal at least 9000 to 10,000 steps to burn adequate amount of calories.
4. If users want to lose weight, it’s probably a good idea to control daily calorie consumption. Bellabeat can suggest some ideas for low-calorie diet or consult health experts through 24*7 customer care service.
5. If users want to improve their sleep, Bellabeat app should give notifications to go to bed.
6. The participants are mostly active during 5 pm to 10 pm. I thing they prefer to go gym or for jogging after office work. Bellabeat app can use this time to remind and motivate them to do excersize.
 Bellabeat company only focused on women's. The company should also focused on men. Company should run some campign about health related and encourage people to use bellabeat smart devices for one stop solution for health.  

**Thank you** for your interest to my Bellabeat Case Study :)

This is my **first project using R**. I would appreciate any comments and recommendations for improvement!